In [2]:
import numpy as np
import pandas as pd
import pyarrow
import random

pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows',500)


rated_bookings_file='./Datasets/rated_bookings.csv'

In [3]:
# rb for Rated Bookings for short typing.
rb=pd.read_csv(rated_bookings_file)